In [2]:
import librosa
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,Model
from keras import layers
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.models import load_model
import numpy as np
import os
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Proprocessing Data
need to run for data prepare and train, test split

In [2]:
# each data has a sequence less than 60000
ENG_DIR='D:\\usc study\\4th semester\\EE599 Deep learning\\EE599_coding\\HW5\\train\\english\\'
HIN_DIR='D:\\usc study\\4th semester\\EE599 Deep learning\\EE599_coding\\HW5\\train\\hindi\\'
CHN_DIR='D:\\usc study\\4th semester\\EE599 Deep learning\\EE599_coding\\HW5\\train\\mandarin\\'

def load_voicedata(num_eng,num_chn,num_hin,length):
    eng_data=np.zeros((num_eng,64,length),dtype=float)
    eng_labels=np.zeros((num_eng,1),dtype=float)
    chn_data=np.zeros((num_chn,64,length),dtype=float)
    chn_labels=np.ones((num_chn,1),dtype=float)*2
    hin_data=np.zeros((num_hin,64,length),dtype=float)
    hin_labels=np.ones((num_hin,1),dtype=float)
    eng_voice = [ENG_DIR+i for i in os.listdir(ENG_DIR)]
    chn_voice = [CHN_DIR+i for i in os.listdir(CHN_DIR)]
    hin_voice = [HIN_DIR+i for i in os.listdir(HIN_DIR)]
    for i,eng_file in enumerate(eng_voice):
        read_data=np.load(eng_file)
        eng_data[i]=read_data[:,:length]
        if i==num_eng-1:
            break
    print('finished preparing english data')
    for i,chn_file in enumerate(chn_voice):
        read_data=np.load(chn_file)
        chn_data[i]=read_data[:,:length]
        if i==num_chn-1:
            break
    print('finished preparing madarin data')
    for i,hin_file in enumerate(hin_voice):
        read_data=np.load(hin_file)
        hin_data[i]=read_data[:,:length]
        if i==num_hin-1:
            break
    print('finished preparing hindi data')
    data=np.concatenate((eng_data,chn_data,hin_data),axis=0)
    labels=np.concatenate((eng_labels,chn_labels,hin_labels),axis=0)
    print('finished preparing data')
    print('data size',data.shape)
    return data,labels
    

In [3]:
data,labels=load_voicedata(60,40,20,20000)

finished preparing english data
finished preparing madarin data
finished preparing hindi data
finished preparing data
data size (120, 64, 20000)


In [4]:
def data_preprocessing(input_data,input_labels):
    data_mean_mid=input_data.mean(axis=0)
    data_mean=data_mean_mid.mean(axis=1)
    data_std_mid=input_data.std(axis=0)
    data_std=data_std_mid.std(axis=1)
    output_data=input_data
    for i in range (0,len(input_data)):
        for j in range (0,input_data.shape[2]):
            output_data[i,:,j]-=data_mean
            output_data[i,:,j]=output_data[i,:,j]/data_std
#    output_data=(input_data-data_mean)/data_std
    output_labels=np.zeros((len(input_data),3),dtype=float)
    for i in range (0,len(input_data)):
        if input_labels[i]==0:
            output_labels[i,0]=1
        elif input_labels[i]==1:
            output_labels[i,1]=1
        else:
            output_labels[i,2]=1
    print('finish data preprocessing, get standard data and one hot labels')
    return output_data,output_labels

In [6]:
all_data,all_labels=data_preprocessing(data,labels)

finish data preprocessing, get standard data and one hot labels


In [7]:
# seperate full length of 50000 sequence into a simple length, and seperate to gene_times apart, try each_length=1000(10s), generate 20 times
def data_generator(input_data,input_labels,each_length,gene_times):
    gene_data=np.zeros((len(input_data)*gene_times,64,each_length),dtype=float)
    gene_labels=np.zeros((len(input_data)*gene_times,3),dtype=float)
    for i in range (0,len(input_data)):
        for j in range (0,gene_times):
            gene_data[i*gene_times+j]=input_data[i,:,j*(each_length+100):j*(each_length+100)+each_length]
            gene_labels[i*gene_times+j]=input_labels[i]
    
    return gene_data,gene_labels
    

In [8]:
gene_data,gene_labels=data_generator(all_data,all_labels,each_length=600,gene_times=25)
gene_data=np.reshape(gene_data,(len(gene_data),600,64))
train_data,val_data,train_labels,val_labels=train_test_split(gene_data,gene_labels,test_size=0.1666)
print('train size:',train_data.shape,'label size:',train_labels.shape)

train size: (2500, 64, 600)


In [21]:
def show_plot():
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

def get_hot_value(prediction_list):
    hot_list=np.zeros((prediction_list.shape[0],prediction_list.shape[1]),int)
    for i in range (0,len(prediction_list)):
        max_val, hot_oneline, max_found = max(prediction_list[i]), [], False
        for item in prediction_list[i]:
            if item == max_val and not max_found:
                hot_oneline.append(1)
                max_found = True
            else:
                hot_oneline.append(0)
        hot_list[i,:]=hot_oneline
    return hot_list

def detail(label,predictions):
    n_right=0
    n_wrong=0
    pred_onehot=get_hot_value(predictions)
    n_wrongclassified=np.zeros((predictions.shape[1]),int) #classified as a label(in predictions), but is not
    n_rightclassified=np.zeros((predictions.shape[1]),int)
    n_allclassified=pred_onehot.sum(axis=0)
    n_alllabeled=label.sum(axis=0)
    for i in range (0,len(predictions)):
        if (label[i]==pred_onehot[i]).all():
            n_right+=1
            n_rightclassified+=pred_onehot[i]
        else:
            n_wrong+=1
            n_wrongclassified+=pred_onehot[i]
    accuracy=n_right/len(predictions)
    return accuracy,n_wrongclassified,n_rightclassified,n_allclassified,n_alllabeled.astype(int)
def performance():
    prediction=model.predict(val_data,verbose=0)
    prediction=prediction.sum()
    accu=detail(val_labels,prediction)
    print('detail of training image')
    print('for test accuracy is %f' %accu[0])
    print('wrong prediction of train label',accu[1])
    print('right prediction of train label',accu[2])
    print('prediction of train label      ',accu[3])
    print('true of train label            ',accu[4])
    show_plot()
    return prediction,accu

# model implement and training
dont't need to run if only test implemented model

In [35]:
def model_LSTM():
    model = Sequential()
    model.add(layers.LSTM(32,
                     dropout=0.1,
                     recurrent_dropout=0.5,
                     return_sequences=True,
                     input_shape=(None,64)))
    model.add(layers.LSTM(4,
                     dropout=0.3,
                     recurrent_dropout=0.5,
                     return_sequences=True))
#    model.add(layers.Flatten())
    model.add(layers.TimeDistributed(layers.Dense(activation='relu', 
                                units=32)))
#    model.add(layers.Dense(16,activation='relu'))
#    model.add(layers.Flatten())
    model.add(layers.Dense(3,activation='softmax'))
#    model.add(Activation('sigmoid'))
    return model 

model=model_LSTM()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, None, 32)          12416     
_________________________________________________________________
lstm_14 (LSTM)               (None, None, 4)           592       
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 32)          160       
_________________________________________________________________
dense_7 (Dense)              (None, None, 3)           99        
Total params: 13,267
Trainable params: 13,267
Non-trainable params: 0
_________________________________________________________________


In [36]:
n_batch=32
n_epoch=10
#train_data_new=train_data.reshape(1200,1,1,64, 600)
train_labels_re=np.reshape(train_labels,(train_labels.shape[0],1,3))
train_data_re=train_data.reshape(train_data.shape[0],train_data.shape[2],train_data.shape[1])
print('input data shape:',train_data_re.shape,'input label shape:',train_labels.shape,'batch size',n_batch)
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy')


early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto') 

history=model.fit(train_data_re,train_labels_re,epochs=n_epoch,batch_size=n_batch,
                    validation_split=0.2,callbacks=[history,early_stopping])

input data shape: (2500, 600, 64) input label shape: (2500, 3)
Train on 2000 samples, validate on 500 samples
Epoch 1/10
2000/2000 [==============================] - 190s 95ms/step - loss: 1.0464 - val_loss: 1.0487
Epoch 2/10
2000/2000 [==============================] - 191s 96ms/step - loss: 1.0172 - val_loss: 1.0436
Epoch 3/10
2000/2000 [==============================] - 192s 96ms/step - loss: 1.0118 - val_loss: 1.0441
Epoch 4/10
2000/2000 [==============================] - 196s 98ms/step - loss: 1.0096 - val_loss: 1.0465
Epoch 00004: early stopping


# test model performance
if need to test uploaded model, uncommit the first line 
> model=load_model('hw5_LSTM.h5')

>detail() function show all performance of test model

In [4]:
#model=load_model('hw5_LSTM.h5')
val_data=np.reshape(val_data,(val_data.shape[0],600,64))
prediction=model.predict(val_data,verbose=0)
prediction=prediction.mean(axis=1)
accu=detail(val_labels,prediction)
print('detail of training image')
print('for test accuracy is %f' %accu[0])
print('wrong prediction of train label',accu[1])
print('right prediction of train label',accu[2])
print('prediction of train label      ',accu[3])
print('true of train label            ',accu[4])
show_plot()

In [23]:
#model.save('hw5_LSTM_new.h5')